#Homework 3

##Gridworld test

In [ ]:
from gridworld_hw3_q1 import GridWorld
import numpy as np
import os


def GridWorld5x5():
    rewards = {
        (2,0): -100,
        (4,4):  100
    }

    walls = [(2,2), (2,3), (2,4), (3,2)]

    g = GridWorld(5, 5, start_position=(0, 0),
            pass_through_reward=0, rewards=rewards, walls=walls)

    return g


if __name__ == '__main__':

    g = GridWorld5x5()

    print(g.world)

    while not g.game_over():
        g.print()
        print()

        actions = g.actions()
        print(actions)

        c = ""
        while c not in actions:
            c = input()

        s, r = g.move(c)

        reward = g.world[s]

        print(s, reward)



#Problem 1

In [ ]:
import numpy as np


#Policy Evaluation
def policy_evaluation(policy, g, gamma=0.9, theta=1e-4, max_iterations=1000):
    V = {s: 0 for s in g.all_states()}
    iteration = 0
    while True:
        delta = 0
        iteration += 1
        for s in g.all_states():
            if s in g.rewards:
                continue  # Skip terminal states

            a = policy[s]  # Get action from policy
            r, c = s
            print(f"Evaluating state {s} with action {a}")

            # Try moving
            next_state, reward = g.move(a)
            print(f"Moved to {next_state}, reward: {reward}")

            # V(s_t+1) policy evaluation
            v_new = reward + gamma * V[next_state]
            delta = max(delta, abs(v_new - V[s]))
            V[s] = v_new

        print(f"Iteration: {iteration}, Max Change: {delta}")

        # Convergence check or too many iterations
        if delta < theta or iteration >= max_iterations:
            print(f"Converged after {iteration} iterations with max delta: {delta}")
            break
    return V


def policy_improvement(V, g, gamma=0.9):
    policy = {}
    for s in g.all_states():
        if s in g.rewards:
            continue  # Terminal states

        best_value = float('-inf')
        best_action = None

        # Get possible actions for current state
        possible_actions = g.actions(s)
        print(f"State {s}: Possible actions: {possible_actions}")

        for a in possible_actions:
            next_state, reward = g.move(a)
            value = reward + gamma * V[next_state]
            print(f"Action {a}: Moving to {next_state}, Value: {value}")
            if value > best_value:
                best_value = value
                best_action = a

        policy[s] = best_action
        print(f"State {s}: Best action {policy[s]} with value {best_value}")
        #Test prints for blocks close to terminal state
        print(f"Value of (3, 3): {V.get((3, 3), 0)}")
        print(f"Value of (4, 4): {V.get((4, 4), 0)}")

    return policy


def policy_iteration(g, gamma=0.9):
    # Initialize policy
    policy = {
        (0, 0): 'R', (0, 1): 'D', (0, 2): 'L', (0, 3): 'L', (0, 4): 'L',
        (1, 0): 'R', (1, 1): 'D', (1, 2): 'L', (1, 3): 'L', (1, 4): 'L',
        (2, 0): 'R', (2, 1): 'D',
        (3, 0): 'D', (3, 1): 'D',              (3, 3): 'R', (3, 4): 'D',
        (4, 0): 'R', (4, 1): 'R', (4, 2): 'R', (4, 3): 'R', (4, 4): 'D',
    }

    while True:
        # Step 1: Policy Evaluation
        print(f"Initial position: {g.r, g.c}")
        V = policy_evaluation(policy, g, gamma)

        # Step 2: Policy Improvement
        new_policy = policy_improvement(V, g, gamma)

        if new_policy == policy:
            break  # Stop if policy is stable
        policy = new_policy

    return policy, V

def print_value_function(V, g):
    # Grid size
    rows, cols = g.rows, g.columns

    # Iterate over rows and columns
    for r in range(rows):
        row_values = []
        for c in range(cols):
            # Get value of the current state (r, c)
            value = V.get((r, c), 0)
            #Cleaner decimal values
            row_values.append(f"{value:6.2f}")
        # Join the row and print it
        print(" | ".join(row_values))
        print("-" * (7 * cols))

if __name__ == '__main__':
    g = GridWorld5x5()

    print("Initial grid layout (self.world):")
    print(g.world)

    policy, V = policy_iteration(g)

    print("Final policy:")
    for r in range(g.rows):
        for c in range(g.columns):
            if (r, c) in policy:
                print(f"{(r, c)}: {policy[(r, c)]}", end=", ")
        print()

    print("Final value function:")
    for r in range(g.rows):
        for c in range(g.columns):
            print(f"{(r, c)}: {V.get((r, c), 0):.2f}", end=", ")
        print()

print("Final value function:")
print_value_function(V, g)

Initial grid layout (self.world):
[[   0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [-100.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.  100.]]
Initial position: (0, 0)
Evaluating state (0, 0) with action R
Moved to (0, 1), reward: 0.0
Evaluating state (0, 1) with action D
Moved to (1, 1), reward: 0.0
Evaluating state (0, 2) with action L
Moved to (1, 0), reward: 0.0
Evaluating state (0, 3) with action L
Moved to (1, 0), reward: 0.0
Evaluating state (0, 4) with action L
Moved to (1, 0), reward: 0.0
Evaluating state (1, 0) with action R
Moved to (1, 1), reward: 0.0
Evaluating state (1, 1) with action D
Moved to (2, 1), reward: 0.0
Evaluating state (1, 2) with action L
Moved to (2, 0), reward: -100.0
Evaluating state (1, 3) with action L
Moved to (2, 0), reward: -100.0
Evaluating state (1, 4) with action L
Moved to (2, 0), reward: -100.0
Evaluating state (2, 1) with action D
Moved to (3, 0), reward: 0.0
Evaluating state (3, 0) with 

#Problem 2

In [ ]:
from gridworld_hw3_q1 import GridWorld
import numpy as np
import os
import random

#From gw55_hw3
def GridWorld5x5(p=0.9):
    rewards = {
        (2,0): -100,
        (4,4):  100
    }
    walls = [(2,2), (2,3), (2,4), (3,2)]

    # deterministic transition table
    T = {
        (0,0): { 'R': (0,1), 'D': (1,0) },
        (0,1): { 'R': (0,2), 'L': (0,0), 'D': (1,1) },
        (0,2): { 'R': (0,3), 'L': (0,1), 'D': (1,2) },
        (0,3): { 'R': (0,4), 'L': (0,2), 'D': (1,3) },
        (0,4): { 'L': (0,3), 'D': (1,4) },

        (1,0): { 'R': (1,1), 'D': (2,0), 'U': (0,0) },
        (1,1): { 'R': (1,2), 'L': (1,0), 'D': (2,1), 'U': (0,1) },
        (1,2): { 'R': (1,3), 'L': (1,1), 'U': (0,2) },
        (1,3): { 'R': (1,4), 'L': (1,2), 'U': (0,3) },
        (1,4): { 'L': (1,3), 'U': (0,4) },

        (2,0): { 'R': (2,1), 'D': (3,0), 'U': (1,0) },
        (2,1): { 'L': (2,0), 'D': (3,1), 'U': (1,1) },

        (3,0): { 'R': (3,1), 'D': (4,0), 'U': (2,0) },
        (3,1): { 'L': (3,0), 'D': (4,1), 'U': (2,1) },
        (3,3): { 'R': (3,4), 'D': (4,3) },
        (3,4): { 'L': (3,3), 'D': (4,4) },

        (4,0): { 'R': (4,1), 'U': (3,0) },
        (4,1): { 'R': (4,2), 'L': (4,0), 'U': (3,1) },
        (4,2): { 'R': (4,3), 'L': (4,1) },
        (4,3): { 'R': (4,4), 'L': (4,2), 'U': (3,3) },
        (4,4): { 'L': (4,3), 'U': (3,4) },
    }

    # convert deterministic transition table to stochastic one
    # where we have probability 'p' of leaving the state and
    # probability '1-p' to remain in the same state.

    # this way it is easier to conver the table above to the
    # stochastic version

    for s in T:
        ns_l = list(T[s].values())
        for a in T[s]:
            ns = T[s][a] #next state
            rs = ns_l[np.random.choice(len(ns_l))] #random
            if ns == rs:
                T[s][a] = { ns: 1.0 }
            else:
                T[s][a] = { ns: p, rs: np.round(1-p,2) }

    g = GridWorld(5, 5, start_position=(0, 0),
            pass_through_reward=0, rewards=rewards, walls = walls)
    g.probs = T

    return g


if __name__ == '__main__':
    g = GridWorld5x5(p=0.5)

    policy = {
        (0, 0): 'R', (0, 1): 'D', (0, 2): 'L', (0, 3): 'L', (0, 4): 'L',
        (1, 0): 'R', (1, 1): 'D', (1, 2): 'L', (1, 3): 'L', (1, 4): 'L',
        (2, 0): 'R', (2, 1): 'D',
        (3, 0): 'D', (3, 1): 'D',              (3, 3): 'R', (3, 4): 'D',
        (4, 0): 'R', (4, 1): 'R', (4, 2): 'R', (4, 3): 'R', (4, 4): 'D',
    }

    print(g.world)

    while not g.game_over():
        g.print()
        print()

        current_state = g.get_state()  # Get current state
        actions = g.actions()  # Get available actions

        # Choose action from policy in current state
        if current_state in policy:
            action = policy[current_state]
        else:
            # random action if no policy for current state
            action = random.choice(actions)

        print(f"Chosen action: {action}")

        # Action moves, gets next state and reward
        next_state, reward = g.move(action)
        print(f"Moved to state: {next_state}")
        print(f"Reward: {reward}")

        # Set the agent's state to the new state
        g.set_state(next_state)


[[   0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [-100.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.  100.]]
+-+-+-+-+-+
|o| | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: R
Moved to state: (0, 1)
Reward: 0.0
+-+-+-+-+-+
| |o| | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Moved to state: (1, 1)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| |o| | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: L
Moved to state: (1, 2)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| | |o| | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: R
Moved to state: (1, 3)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| | | |o| |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | 

#Problem 3

In [ ]:
import numpy as np

def policy_evaluation(policy, g, gamma=0.9, theta=1e-4, max_iterations=1000):
    V = {s: 0 for s in g.all_states()}  # Initialize value function
    iteration = 0
    while True:
        delta = 0
        iteration += 1
        for s in g.all_states():
            if s in g.rewards:  # Skip terminal states
                continue

            a = policy[s]  # Get the action from the current policy
            next_states_probs = g.probs.get(s, {}).get(a, {})  # Get possible next states and their probabilities
            v_new = 0
            for next_state, prob in next_states_probs.items():
                reward = g.world[next_state]  # Get the reward for the next state
                v_new += prob * (reward + gamma * V[next_state])

            delta = max(delta, abs(v_new - V[s]))
            V[s] = v_new

        if delta < theta or iteration >= max_iterations:
            break
    return V

def policy_improvement(V, g, gamma=0.9):
    policy = {}
    for s in g.all_states():
        if s in g.rewards:  # Skip terminal states
            continue

        best_value = float('-inf')
        best_action = None

        possible_actions = g.actions(s)
        for a in possible_actions:
            next_states_probs = g.probs.get(s, {}).get(a, {})
            value = 0
            for next_state, prob in next_states_probs.items():
                reward = g.world[next_state]
                value += prob * (reward + gamma * V[next_state])

            if value > best_value:
                best_value = value
                best_action = a

        policy[s] = best_action
    return policy

def policy_iteration(g, gamma=0.9):
    # Initialize a random policy
    policy = {}
    for s in g.all_states():
        possible_actions = g.actions(s)
        if possible_actions:
            policy[s] = np.random.choice(possible_actions)  # Randomly assign an action to each state

    while True:
        # Step 1: Policy Evaluation
        V = policy_evaluation(policy, g, gamma)

        # Step 2: Policy Improvement
        new_policy = policy_improvement(V, g, gamma)

        if new_policy == policy:
            break  # Stop if policy is stable
        policy = new_policy

    return policy, V


# Run policy iteration on the gridworld
if __name__ == '__main__':
    g = GridWorld5x5(p=0.5)

    policy, V = policy_iteration(g)



    while not g.game_over():
        g.print()  # Print the gridworld state
        print()

        current_state = g.get_state()  # Get current state
        actions = g.actions()  # Get available actions

        # Choose action from policy for current state
        if current_state in policy:
            action = policy[current_state]
        else:
            # Random action if no policy for current shape
            action = random.choice(actions)

        print(f"Chosen action: {action}")

        # Move action and gets next state and reward
        next_state, reward = g.move(action)
        print(f"Moved to state: {next_state}")
        print(f"Reward: {reward}")

        # Set the agent's state to the new state
        g.set_state(next_state)

    print("Optimal Policy:")
    for r in range(g.rows):
        for c in range(g.columns):
            if (r, c) in policy:
                print(f"{(r, c)}: {policy[(r, c)]}", end=", ")
        print()

    print("Final Value Function:")
    for r in range(g.rows):
        for c in range(g.columns):
            print(f"{(r, c)}: {V.get((r, c), 0):.2f}", end=", ")
        print()


+-+-+-+-+-+
|o| | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: R
Moved to state: (0, 1)
Reward: 0.0
+-+-+-+-+-+
| |o| | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Moved to state: (1, 1)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| |o| | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Moved to state: (2, 1)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B|o|x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Moved to state: (3, 1)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| |o|x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Moved to state: (2, 1)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B|o|x|x|x|
+-+

#Problem 4

In [ ]:
def value_iteration(g, gamma=0.9, theta=1e-4, max_iterations=1000):
    V = {s: 0 for s in g.all_states()}  # Initialize value function
    iteration = 0

    while True:
        delta = 0
        iteration += 1
        for s in g.all_states():
            if s in g.rewards:  # Skip terminal states
                continue

            best_value = float('-inf')
            possible_actions = g.actions(s)

            # V(s) equation for each action
            for a in possible_actions:
                next_states_probs = g.probs.get(s, {}).get(a, {})
                value = 0
                for next_state, prob in next_states_probs.items():
                    reward = g.world[next_state]
                    value += prob * (reward + gamma * V[next_state])

                best_value = max(best_value, value)

            delta = max(delta, abs(best_value - V[s]))
            V[s] = best_value

        if delta < theta or iteration >= max_iterations:
            break

    # Extract optimal policy based on value function
    policy = {}
    for s in g.all_states():
        if s in g.rewards:  # Skip terminal states
            continue

        best_action = None
        best_value = float('-inf')
        possible_actions = g.actions(s)
        for a in possible_actions:
            next_states_probs = g.probs.get(s, {}).get(a, {})
            value = 0
            for next_state, prob in next_states_probs.items():
                reward = g.world[next_state]
                value += prob * (reward + gamma * V[next_state])

            if value > best_value:
                best_value = value
                best_action = a

        policy[s] = best_action

    return policy, V


# Run value iteration on the gridworld
if __name__ == '__main__':
    g = GridWorld5x5(p=0.5)

    policy, V = value_iteration(g)

    while not g.game_over():
        g.print()  # Print the gridworld state
        print()

        current_state = g.get_state()  # Get current state
        actions = g.actions()  # Get available actions

        # Choose action from policy for current state
        if current_state in policy:
            action = policy[current_state]
        else:
            # Random action if no policy defined for currnet state
            action = random.choice(actions)

        print(f"Chosen action: {action}")

        # Action moves and gets next state and reward
        next_state, reward = g.move(action)
        print(f"Moved to state: {next_state}")
        print(f"Reward: {reward}")

        # Set the agent's state to the new state
        g.set_state(next_state)

    print("Optimal Policy:")
    for r in range(g.rows):
        for c in range(g.columns):
            if (r, c) in policy:
                print(f"{(r, c)}: {policy[(r, c)]}", end=", ")
        print()

    print("Final Value Function:")
    for r in range(g.rows):
        for c in range(g.columns):
            print(f"{(r, c)}: {V.get((r, c), 0):.2f}", end=", ")
        print()


+-+-+-+-+-+
|o| | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Moved to state: (1, 0)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|o| | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: U
Moved to state: (0, 0)
Reward: 0.0
+-+-+-+-+-+
|o| | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: D
Moved to state: (1, 0)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|o| | | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: U
Moved to state: (1, 1)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| |o| | | |
+-+-+-+-+-+
|B| |x|x|x|
+-+-+-+-+-+
| | |x| | |
+-+-+-+-+-+
| | | | |G|
+-+-+-+-+-+

Chosen action: R
Moved to state: (2, 1)
Reward: 0.0
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
| | | | | |
+-+-+-+-+-+
|B|o|x|x|x|
+-+